In [233]:
#falta definir el tema de periodos 

In [1]:
from dbconn import querydb, querydbtopandas 
import numpy as np
import pandas as pd

In [243]:
pd.set_option('display.max_columns', 500)

In [318]:
#esto solo tiene en cuenta febrero 
mayor = "2019-03"
menor = "2019-02"

In [341]:
df = querydbtopandas(f"""select    a11.id_pres_prestacion  id_pres_prestacion,

                max(a17.desc_pres_prestacion)  desc_pres_prestacion,

                max(a17.prestac_pres_prestacion)  prestac_pres_prestacion,

                a11.id_nom_nomenclador  id_nom_nomenclador,

                a11.id_pre_prestador  id_pre_prestador,

                max(a15.desc_pre_nombre)  desc_pre_nombre,

                max(a15.vigente)  vigente,

                a12.id_pre_prestador  id_pre_prestador0,

                max(a16.desc_pre_nombre)  desc_pre_nombre0,

                sum(a11.i_ft_importe_prestacion)  RDENES,

                sum(case  when (  i_ft_importe_prestacion *i_Ft_Cantidad <= 0) then 0  when  (i_ft_importe_prestacion *i_Ft_Cantidad <> 0 ) then   i_Ft_Cantidad   end )  CANTIDADORDENES0

from      DBA.ft_ordenes               a11

                join        DBA.d_auto_autorizacion     a12

                  on        (((a11.id_auto_sucursal * 10000000) + a11.i_ft_autorizacion) = ((a12.id_auto_sucursal * 10000000) + a12.id_auto_autorizacion))

                join        DBA.d_exp_expediente a13

                  on        (((a11.id_ofi_oficina * 10000000) + a11.id_exp_expediente) = ((a13.ofi * 10000000) + a13.id_exp_expediente))

                join        DBA.fecha_mes a14

                  on        (a13.exp_mes = a14.exp_mes)

                join        DBA.d_pre_prestador   a15

                  on        (a11.id_pre_prestador = a15.id_pre_prestador)

                join        DBA.d_pre_prestador_autoriza a16

                  on        (a12.id_pre_prestador = a16.id_pre_prestador)

                join        DBA.d_pres_prestacion a17

                  on        (a11.id_pres_prestacion = a17.id_pres_prestacion)

where (CONVERT(DATETIME,a13.exp_mes, "mm-YYYY") >= date("{menor}") and CONVERT(DATETIME,a13.exp_mes, "mm-YYYY") < date("{mayor}")

and a15.id_pre_tipo in ('E','P')

and a11.id_nom_nomenclador in (1,11)

and (not a11.id_pre_prestador in (154688)))

group by             a11.id_pres_prestacion,

                a11.id_nom_nomenclador,

                a11.id_pre_prestador,

                a12.id_pre_prestador""")

In [342]:
df

,id_pres_prestacion,desc_pres_prestacion,prestac_pres_prestacion,id_nom_nomenclador,id_pre_prestador,desc_pre_nombre,vigente,id_pre_prestador0,desc_pre_nombre0,RDENES,CANTIDADORDENES0
0,48,Descompresión del mediano a nivel del túnel ca...,010505,1,26559,"Petraglia Ramos,Guillermo L.",A,1539,Sanatorio Mater Dei,1846.67,1.0
1,76,"Blefaroplastía -injerto, cantoplastía, trasopl...",020202,1,56921,"Sereday,Carlos",A,182730,CLINICA ZABALA,5559.43,1.0
2,79,"Escisión de lesión de párpados, blefarectomía ...",020205,1,44665,"Ghilino,Oscar D.",A,44665,"Ghilino,Oscar D.",2531.04,1.0
3,81,Escision de lesion conjuntiva -quiste epitelio...,020302,1,56921,"Sereday,Carlos",A,56921,"Sereday,Carlos",3123.62,1.0
4,81,Escision de lesion conjuntiva -quiste epitelio...,020302,1,80593,"Lavalle Paniza,Alfredo J.",A,80593,"Lavalle Paniza,Alfredo J.",4938.08,1.0
...,...,...,...,...,...,...,...,...,...,...,...
1799,35833,Cirugía de Cadera con prótesis de Banco.,12172619,11,20203,"Copquin,Alberto D.",A,99785,SANATORIO DE LOS ARCOS,7678.24,1.0
1800,35833,Cirugía de Cadera con prótesis de Banco.,12172619,11,144974,"Sayago,Gustavo A.",A,0,Sin Asignar,7678.24,1.0
1801,35833,Cirugía de Cadera con prótesis de Banco.,12172619,11,144974,"Sayago,Gustavo A.",A,99785,SANATORIO DE LOS ARCOS,30712.96,4.0
1802,36102,Masto plastia reductora Post Cirugía de la Obe...,13021146,11,20199,"Sokolowicz,Daniel",A,1763,CLINICA Y MATERNIDAD SUIZO ARGENTINA,21971.65,1.0


In [343]:
df = df[df["id_pre_prestador0"] != 0] 

In [344]:
len(df)

1499

In [345]:
cirujias_validas = pd.read_csv("cx_evaluar.csv",dtype=str, header=None)

In [346]:
cirujias_validas = cirujias_validas[0].tolist()

In [347]:
df["prestac_pres_prestacion"] = df["prestac_pres_prestacion"].str.strip()

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [348]:
df = df[df["prestac_pres_prestacion"].isin(cirujias_validas_str)]

In [349]:
centros_propios = [1632,1763,
2086,26564,31244,55060,64677,75470,78363,85316,85401,99785,118209,122803,135464,182730,187243,196559,196561,201019,206939,210252,211421,213729,217747,226756,235779,240895]

In [350]:
df_propios = df[df["id_pre_prestador0"].isin(centros_propios)]

In [351]:
df_no_propios = df[~df["id_pre_prestador0"].isin(centros_propios)]

In [352]:
propios = df_propios.groupby(["id_pre_prestador", "desc_pre_nombre"])["CANTIDADORDENES0"].sum().reset_index().rename(columns = {"CANTIDADORDENES0": "propios"})

In [353]:
no_propios = df_no_propios.groupby(["id_pre_prestador", "desc_pre_nombre"])["CANTIDADORDENES0"].sum().reset_index().rename(columns = {"CANTIDADORDENES0": "no_propios"})

In [354]:
final = pd.merge(propios, no_propios,how="outer")

In [355]:
final.fillna(0,inplace=True)

In [356]:
final["total"]  = final["propios"] + final["no_propios"]

In [357]:
final["porcentaje en propios"] = (final["propios"] /final["total"])

In [358]:
final = final.round()

In [359]:
final.rename(columns= {"id_pre_prestador": "Prestador", "desc_pre_nombre": "Razón Social", "propios": "Propias", "no_propios": "No propias", "total": "Total general","porcentaje en propios":"% en Propias" },inplace=True)

In [360]:
final

,Prestador,Razón Social,Propias,No propias,Total general,% en Propias
0,172,"Pasqualini,Rodolfo S.",12.0,2.0,14.0,1.0
1,193,"Ferro,Paula C.",1.0,0.0,1.0,1.0
2,423,"Pérez Macri,Silvia C.",4.0,0.0,4.0,1.0
3,438,"Ortega Traverso,Jorge A.",3.0,0.0,3.0,1.0
4,509,"Montesinos,Manuel R.",1.0,0.0,1.0,1.0
...,...,...,...,...,...,...
547,205475,"Martins,Andrea E.",0.0,2.0,2.0,0.0
548,205825,"Veglia,María Florencia",0.0,2.0,2.0,0.0
549,209138,"Perea Ansinelli,Alexis G.",0.0,4.0,4.0,0.0
550,211271,"Lucarelli,Pablo",0.0,4.0,4.0,0.0


In [230]:
final_50_febrebro  = final.head(50)

In [232]:
final_50_febrebro.to_excel("final_febrero_50_prestadores_mas_a_verificar_final_final.xlsx")

In [291]:
pd.read_excel("final_febrero_50_prestadores_mas_a_verificar_final_final.xlsx")

,Unnamed: 0,Prestador,Razón Social,Propias,No propias,Total general,% en Propias
0,0,172,"Pasqualini,Rodolfo S.",12,2,14,0.857143
1,1,193,"Ferro,Paula C.",1,0,1,1.000000
2,2,423,"Pérez Macri,Silvia C.",4,0,4,1.000000
3,3,438,"Ortega Traverso,Jorge A.",3,0,3,1.000000
4,4,509,"Montesinos,Manuel R.",1,0,1,1.000000
5,5,624,"Trepat,Ana M.",2,0,2,1.000000
6,6,625,"Muñiz,Alcira A.",1,0,1,1.000000
7,7,632,"Lavarello,Marcelo",2,0,2,1.000000
8,8,715,"Bistoletti,Pedro H.",2,0,2,1.000000
9,9,744,"Vigliocco,Guillermo",7,0,7,1.000000
